In [ ]:
import struct
import sys

def reverse_search_moov_atom(filepath):
    try:
        with open(filepath, 'rb') as file:
            file.seek(0, 2)  # 파일의 끝으로 이동
            file_size = file.tell()
            step_size = 1024  # 한 번에 읽을 바이트 수
            current_step = 0

            while file_size - current_step > 0:
                start = file_size - current_step - step_size
                start = max(start, 0)  # 시작점이 파일 시작보다 앞서지 않도록 조정
                file.seek(start)
                data = file.read(min(step_size, file_size - current_step))

                # 'moov' 문자열을 역순으로 찾습니다.
                index = data[::-1].find(b'voom')  # 'moov'를 역순으로 검색
                if index != -1:
                    # 'moov' 아톰의 시작 위치를 계산합니다.
                    moov_start = start + step_size - index - 4  # 'moov' 문자열의 시작 위치
                    file.seek(moov_start - 4)  # 아톰 크기 정보 위치로 이동
                    moov_size = struct.unpack('>I', file.read(4))[0]  # 'moov' 아톰의 크기를 읽음
                    return moov_start - 4, moov_size
                current_step += step_size

        return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def find_bkmk_in_moov(filepath, moov_position, moov_size):
    bkmk_positions = []
    try:
        with open(filepath, 'rb') as file:
            file.seek(moov_position)  # 'moov' 아톰의 시작 위치로 이동
            moov_data = file.read(moov_size)  # 'moov' 아톰의 전체 데이터를 읽음

            offset = 0
            while offset < len(moov_data):
                index = moov_data[offset:].find(b'bkmk')
                if index == -1:
                    break  # 더 이상 'bkmk'를 찾을 수 없으면 종료
                bkmk_position = moov_position + offset + index
                bkmk_positions.append(bkmk_position)
                offset += index + 4  # 다음 검색을 위해 오프셋 업데이트

        return bkmk_positions
    except Exception as e:
        print(f"Error: {e}")
        return []

def parse_moov_atom_and_find_bkmk(filepath):
    moov_position, moov_size = reverse_search_moov_atom(filepath)
    if moov_position is not None and moov_size is not None:
        print(f"'moov' 아톰 위치: {moov_position}, 크기: {moov_size}")
        bkmk_positions = find_bkmk_in_moov(filepath, moov_position, moov_size)
        if bkmk_positions:
            print(f"'bkmk' 키워드 위치: {bkmk_positions}")
        else:
            print("모옴 내에서 'bkmk' 키워드를 찾을 수 없습니다.")
    else:
        print("파일에서 'moov' 아톰을 찾을 수 없습니다.")

# 파일 경로를 지정합니다. 예: 'example.m4a'
filepath = 'TestoriD.m4a'
parse_moov_atom_and_find_bkmk(filepath)
